# Loop 14 Analysis: Fix Overlap in N=89

## Problem
exp_013 failed with "Overlapping trees in group 089"

## Strategy
1. Identify which source provided N=89 solution
2. Fall back to baseline for N=89
3. Create new submission with stricter validation

In [1]:
import sys
sys.path.insert(0, '/home/code')

import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon

from code.tree_geometry import TX, TY, calculate_score, get_tree_vertices_numba
from code.utils import parse_submission

getcontext().prec = 30
SCALE = 10**18

# Load the failed submission
failed_df = pd.read_csv('/home/code/experiments/013_selective_threshold/submission.csv')
failed_configs = parse_submission(failed_df)

# Load baseline
baseline_df = pd.read_csv('/home/code/experiments/001_valid_baseline/submission.csv')
baseline_configs = parse_submission(baseline_df)

# Load exp_010 (last successful submission)
exp010_df = pd.read_csv('/home/code/experiments/010_safe_ensemble/submission.csv')
exp010_configs = parse_submission(exp010_df)

print("Loaded all submissions")

Loaded all submissions


In [2]:
def get_tree_polygon_highprec(x, y, angle_deg):
    """Get tree polygon with high-precision integer coordinates."""
    rx, ry = get_tree_vertices_numba(x, y, angle_deg)
    coords = []
    for xi, yi in zip(rx, ry):
        xi_int = int(Decimal(str(xi)) * SCALE)
        yi_int = int(Decimal(str(yi)) * SCALE)
        coords.append((xi_int, yi_int))
    return Polygon(coords)

def validate_no_overlap_strict(trees):
    """Validate no overlaps using integer arithmetic."""
    if len(trees) <= 1:
        return True, None
    
    polygons = []
    for x, y, angle in trees:
        poly = get_tree_polygon_highprec(x, y, angle)
        if not poly.is_valid:
            return False, "Invalid polygon"
        polygons.append(poly)
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                if not polygons[i].touches(polygons[j]):
                    inter = polygons[i].intersection(polygons[j])
                    if inter.area > 0:
                        return False, f"Trees {i} and {j} overlap with area {inter.area}"
    return True, None

# Check N=89 in failed submission
n = 89
config = failed_configs[n]
valid, msg = validate_no_overlap_strict(config)
print(f"N={n} in failed submission: valid={valid}, msg={msg}")
print(f"Score: {calculate_score(config):.6f}")

N=89 in failed submission: valid=True, msg=None


Score: 0.346387


In [3]:
# Check N=89 in baseline
config_baseline = baseline_configs[n]
valid_baseline, msg_baseline = validate_no_overlap_strict(config_baseline)
print(f"N={n} in baseline: valid={valid_baseline}, msg={msg_baseline}")
print(f"Score: {calculate_score(config_baseline):.6f}")

# Check N=89 in exp_010
config_exp010 = exp010_configs[n]
valid_exp010, msg_exp010 = validate_no_overlap_strict(config_exp010)
print(f"N={n} in exp_010: valid={valid_exp010}, msg={msg_exp010}")
print(f"Score: {calculate_score(config_exp010):.6f}")

N=89 in baseline: valid=True, msg=None
Score: 0.346717
N=89 in exp_010: valid=True, msg=None
Score: 0.346717


In [4]:
# Compare the configurations
print("\nComparing N=89 configurations:")
print(f"Failed submission score: {calculate_score(failed_configs[n]):.6f}")
print(f"Baseline score: {calculate_score(baseline_configs[n]):.6f}")
print(f"exp_010 score: {calculate_score(exp010_configs[n]):.6f}")

# Check if failed config is different from baseline/exp010
failed_str = str(failed_configs[n])
baseline_str = str(baseline_configs[n])
exp010_str = str(exp010_configs[n])

print(f"\nFailed == Baseline: {failed_str == baseline_str}")
print(f"Failed == exp_010: {failed_str == exp010_str}")
print(f"Baseline == exp_010: {baseline_str == exp010_str}")


Comparing N=89 configurations:
Failed submission score: 0.346387
Baseline score: 0.346717
exp_010 score: 0.346717

Failed == Baseline: False
Failed == exp_010: False
Baseline == exp_010: True


In [5]:
# Find all N values where failed submission differs from exp_010
print("\nN values where failed submission differs from exp_010:")
different_n = []
for n in range(1, 201):
    failed_score = calculate_score(failed_configs[n])
    exp010_score = calculate_score(exp010_configs[n])
    if abs(failed_score - exp010_score) > 1e-9:
        different_n.append((n, failed_score, exp010_score, exp010_score - failed_score))
        
print(f"Total different N values: {len(different_n)}")
for n, f_score, e_score, diff in different_n[:20]:
    print(f"  N={n}: failed={f_score:.6f}, exp010={e_score:.6f}, diff={diff:.6f}")


N values where failed submission differs from exp_010:
Total different N values: 53
  N=15: failed=0.376382, exp010=0.376949, diff=0.000566
  N=29: failed=0.366573, exp010=0.367082, diff=0.000510
  N=35: failed=0.365937, exp010=0.366057, diff=0.000120
  N=36: failed=0.357372, exp010=0.358047, diff=0.000674
  N=43: failed=0.366822, exp010=0.367065, diff=0.000243
  N=47: failed=0.356616, exp010=0.357493, diff=0.000877
  N=49: failed=0.362683, exp010=0.363430, diff=0.000747
  N=50: failed=0.360004, exp010=0.360753, diff=0.000749
  N=57: failed=0.353112, exp010=0.353508, diff=0.000396
  N=63: failed=0.352638, exp010=0.352794, diff=0.000156
  N=64: failed=0.348540, exp010=0.348685, diff=0.000145
  N=68: failed=0.353515, exp010=0.353622, diff=0.000107
  N=69: failed=0.353583, exp010=0.354528, diff=0.000946
  N=78: failed=0.350652, exp010=0.351401, diff=0.000749
  N=87: failed=0.348990, exp010=0.349960, diff=0.000969
  N=88: failed=0.347380, exp010=0.347501, diff=0.000120
  N=89: failed=0.34

In [6]:
# Check if N=89 is in the different list
for n, f_score, e_score, diff in different_n:
    if n == 89:
        print(f"N=89 found in different list: failed={f_score:.6f}, exp010={e_score:.6f}, diff={diff:.6f}")
        break
else:
    print("N=89 NOT in different list - same as exp_010")

N=89 found in different list: failed=0.346387, exp010=0.346717, diff=0.000330


In [7]:
# The issue is that N=89 passed local validation but failed Kaggle's
# This means Kaggle uses even stricter validation
# Solution: Add N=89 to the problematic list and rerun

print("\n" + "="*70)
print("SOLUTION: Add N=89 to problematic N values")
print("="*70)
print("\nNew problematic N values: {2, 70, 79, 89, 123, 138}")
print("\nThis will use the higher threshold (0.001) for N=89")
print("Or we can simply fall back to exp_010 for N=89")


SOLUTION: Add N=89 to problematic N values

New problematic N values: {2, 70, 79, 89, 123, 138}

This will use the higher threshold (0.001) for N=89
Or we can simply fall back to exp_010 for N=89


In [8]:
# Let's check which N values in the failed submission might have issues
# by comparing with exp_010 (which passed Kaggle validation)

print("\nChecking all N values for potential overlap issues...")
print("(Comparing failed submission with exp_010)")

potential_issues = []
for n in range(1, 201):
    failed_config = failed_configs[n]
    exp010_config = exp010_configs[n]
    
    # Check if they're different
    failed_score = calculate_score(failed_config)
    exp010_score = calculate_score(exp010_config)
    
    if abs(failed_score - exp010_score) > 1e-9:
        # This N value is different - check for overlaps
        valid, msg = validate_no_overlap_strict(failed_config)
        if not valid:
            potential_issues.append((n, failed_score, exp010_score, msg))
            print(f"  N={n}: OVERLAP DETECTED - {msg}")

print(f"\nTotal N values with detected overlaps: {len(potential_issues)}")


Checking all N values for potential overlap issues...
(Comparing failed submission with exp_010)



Total N values with detected overlaps: 0


In [9]:
# Since our local validation passes but Kaggle fails,
# the safest approach is to fall back to exp_010 for ALL N values
# that are different from exp_010

print("\n" + "="*70)
print("SAFEST APPROACH: Use exp_010 as base, only keep improvements that are LARGE")
print("="*70)

# Calculate what we'd get if we only kept improvements > 0.001
from code.utils import save_submission

MIN_IMPROVEMENT = 0.001

best_per_n = {}
improvements = []

for n in range(1, 201):
    exp010_score = calculate_score(exp010_configs[n])
    failed_score = calculate_score(failed_configs[n])
    
    improvement = exp010_score - failed_score
    
    if improvement >= MIN_IMPROVEMENT:
        # Check overlap
        valid, msg = validate_no_overlap_strict(failed_configs[n])
        if valid:
            best_per_n[n] = failed_configs[n]
            improvements.append((n, improvement))
        else:
            best_per_n[n] = exp010_configs[n]
    else:
        best_per_n[n] = exp010_configs[n]

print(f"\nN values with improvements >= {MIN_IMPROVEMENT}: {len(improvements)}")
for n, imp in improvements[:10]:
    print(f"  N={n}: improvement={imp:.6f}")

final_score = sum(calculate_score(best_per_n[n]) for n in range(1, 201))
print(f"\nFinal score: {final_score:.6f}")
print(f"exp_010 score: {sum(calculate_score(exp010_configs[n]) for n in range(1, 201)):.6f}")


SAFEST APPROACH: Use exp_010 as base, only keep improvements that are LARGE

N values with improvements >= 0.001: 0

Final score: 70.365091
exp_010 score: 70.365091


In [10]:
# The issue is that even with MIN_IMPROVEMENT=0.001, N=89 might still fail
# Let's check if N=89 has improvement >= 0.001

n = 89
exp010_score = calculate_score(exp010_configs[n])
failed_score = calculate_score(failed_configs[n])
improvement = exp010_score - failed_score

print(f"N=89: exp010={exp010_score:.6f}, failed={failed_score:.6f}, improvement={improvement:.6f}")
print(f"Improvement >= 0.001: {improvement >= 0.001}")

N=89: exp010=0.346717, failed=0.346387, improvement=0.000330
Improvement >= 0.001: False


In [11]:
# The real issue is that Kaggle's validation is stricter than ours
# We need to be MORE conservative

# Strategy: Only keep improvements that are VERY large (> 0.005)
# AND pass our strictest validation

print("\n" + "="*70)
print("ULTRA-CONSERVATIVE APPROACH")
print("="*70)

MIN_IMPROVEMENT = 0.005  # Very conservative

best_per_n = {}
improvements = []

for n in range(1, 201):
    exp010_score = calculate_score(exp010_configs[n])
    failed_score = calculate_score(failed_configs[n])
    
    improvement = exp010_score - failed_score
    
    if improvement >= MIN_IMPROVEMENT:
        # Check overlap
        valid, msg = validate_no_overlap_strict(failed_configs[n])
        if valid:
            best_per_n[n] = failed_configs[n]
            improvements.append((n, improvement))
        else:
            best_per_n[n] = exp010_configs[n]
    else:
        best_per_n[n] = exp010_configs[n]

print(f"\nN values with improvements >= {MIN_IMPROVEMENT}: {len(improvements)}")
for n, imp in improvements:
    print(f"  N={n}: improvement={imp:.6f}")

final_score = sum(calculate_score(best_per_n[n]) for n in range(1, 201))
print(f"\nFinal score: {final_score:.6f}")
print(f"exp_010 score: {sum(calculate_score(exp010_configs[n]) for n in range(1, 201)):.6f}")
print(f"Improvement over exp_010: {sum(calculate_score(exp010_configs[n]) for n in range(1, 201)) - final_score:.6f}")


ULTRA-CONSERVATIVE APPROACH

N values with improvements >= 0.005: 0

Final score: 70.365091
exp_010 score: 70.365091
Improvement over exp_010: 0.000000


In [ ]:
# Let's understand the gap better
# Current best: 70.365091 (exp_010)
# Target: 68.878195
# Gap: 1.49 points (2.1%)

# What would it take to close this gap?
# The score is sum of (side^2 / n) for n=1 to 200

# Let's analyze the per-N scores
import sys
sys.path.insert(0, '/home/code')

import pandas as pd
import numpy as np
from code.tree_geometry import calculate_score
from code.utils import parse_submission

# Load exp_010
exp010_df = pd.read_csv('/home/code/experiments/010_safe_ensemble/submission.csv')
exp010_configs = parse_submission(exp010_df)

# Calculate per-N scores
per_n_scores = {}
for n in range(1, 201):
    per_n_scores[n] = calculate_score(exp010_configs[n])

total = sum(per_n_scores.values())
print(f"Total score: {total:.6f}")
print(f"Target: 68.878195")
print(f"Gap: {total - 68.878195:.6f}")

# What's the average improvement needed per N?
avg_improvement_needed = (total - 68.878195) / 200
print(f"\nAverage improvement needed per N: {avg_improvement_needed:.6f}")

# What percentage improvement is needed?
pct_improvement = (total - 68.878195) / total * 100
print(f"Percentage improvement needed: {pct_improvement:.2f}%")